<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

> a. https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz & https://www.d.umn.edu/~tpederse/Data/README.int.txt

> b. Le fichier interest-original.txt est un fichier texte brut. Les occurrences du mot "interest" sont annotées avec un suffixe de type "_X", où X est un nombre représentant le sens spécifique du mot. Par exemple, "interest_6" et "interest_5" indiquent deux sens différents.

> c. Oui il y a des occurrences au pluriel ("interests") dans le fichier, également annotées avec un suffixe pour indiquer leur sens.

> d. Les phrases avec plusieurs occurrences de "interest" ou "interests" annotent chaque occurrence séparément, chaque mot ayant son propre identifiant de sens par exemple "interest_6" et "interest_5" dans la même phrase.

**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

 - Sense 1 : Readiness to give attention

 - Sense 2 : Quality of causing attention to be given to

 - Sense 3 : Activity, etc. that one gives attention to

 - Sense 4 : Advantage, advancement, or favor

 - Sense 5 : A share in a company or business

 - Sense 6 : Money paid for the use of money

**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

Les six sens annotés du mot interest dans le corpus proviennent de la première édition électronique du Longman Dictionary of Contemporary English (LDOCE). Ce dictionnaire est accessible en ligne à l'adresse suivante : https://www.ldoceonline.com/

- Sense 1 = "the feeling of wanting to give your attention to something or of wanting to be involved with and to discover more about something"

- Sense 2 = "the quality that makes you think that something is interesting"

- Sense 3 = "an activity or subject that you enjoy and spend time doing or studying"

- Sense 4 = "something that brings advantages to someone or something; a benefit or advantage"

- Sense 5 = "a legal share in a business, property, or other financial venture"

- Sense 6 = "money that is charged by a bank or other financial organization for borrowing money, or paid to someone for investing money"

**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

- Sense 1 : Readiness to give attention

Définition WordNet : a sense of concern with and curiosity about someone or something

- Sense 2 : Quality of causing attention to be given to

Définition WordNet : the power of attracting or holding one's attention (because it is unusual or exciting etc.)

- Sense 3 : Activity, etc. that one gives attention to

Définition WordNet : a diversion that occupies one's time and thoughts (usually pleasantly)

- Sense 4 : Advantage, advancement or favor

Définition WordNet : a reason for wanting something done

- Sense 5 : A share in a company or business

Définition WordNet : a right or legal share of something; a financial involvement with something

- Sense 6 : Money paid for the use of money

Définition WordNet : a fixed charge for borrowing money; usually a percentage of the amount borrowed

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [5]:
import nltk

In [6]:
from random import randrange

In [7]:
raw_definitions = [
    "readiness to give attention",
    "quality of causing attention to be given to",
    "activity, etc. that one gives attention to",
    "advantage, advancement or favor",
    "a share in a company or business",
    "money paid for the use of money"
]

stopwords = {"to", "of", "and", "or", "for", "in", "the", "a", "be", "etc.", "that", "one", "is", "on", "with"}

senses1 = [
    word.lower() 
    for definition in raw_definitions 
    for word in definition.split() 
    if len(word) >= 4 and word.lower() not in stopwords
]

print(senses1)

['readiness', 'give', 'attention', 'quality', 'causing', 'attention', 'given', 'activity,', 'gives', 'attention', 'advantage,', 'advancement', 'favor', 'share', 'company', 'business', 'money', 'paid', 'money']


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [9]:
import re

In [17]:
definitions_combined = [
    # Sense 1
    "readiness to give attention wanting to know or learn about something or someone an activity that you enjoy doing or a subject that you enjoy studying",
    
    # Sense 2
    "quality of causing attention to be given to the quality of attracting interest or attention",
    
    # Sense 3
    "activity that one gives attention to a real or apparent aim or purpose a topic or subject of concern or curiosity",
    
    # Sense 4
    "advantage advancement or favor advantage or profit interest in the success of a business",
    
    # Sense 5
    "a share in a company or business a legal concern or right a right to receive part of the proceeds of a loan or investment",
    
    # Sense 6
    "money paid for the use of money a charge for the use of money usually a percentage of the amount borrowed money paid regularly at a particular rate for the use of money lent"
]

# Fonction pour extraire mots > 3 lettres et uniques
def extract_keywords(text):
    # Met en minuscule et extrait mots alpha >= 4 lettres
    words = re.findall(r'\b[a-z]{4,}\b', text.lower())
    return list(set(words))

senses2 = [extract_keywords(defn) for defn in definitions_combined]

print(senses2)

[['attention', 'activity', 'someone', 'that', 'know', 'give', 'learn', 'doing', 'subject', 'studying', 'enjoy', 'readiness', 'about', 'wanting', 'something'], ['attention', 'causing', 'given', 'quality', 'interest', 'attracting'], ['curiosity', 'attention', 'topic', 'concern', 'that', 'purpose', 'real', 'apparent', 'subject', 'gives', 'activity'], ['advantage', 'advancement', 'interest', 'favor', 'profit', 'success', 'business'], ['part', 'concern', 'receive', 'share', 'company', 'loan', 'proceeds', 'investment', 'right', 'business', 'legal'], ['money', 'percentage', 'amount', 'particular', 'regularly', 'lent', 'usually', 'rate', 'charge', 'borrowed', 'paid']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [14]:
filepath = "interest-original.txt"

sentences = []

with open(filepath, 'r', encoding='utf-8') as file:
    content = file.read()

raw_sentences = content.split('$$')

for sent in raw_sentences:
    sent = sent.strip()
    if sent == "" or sent.startswith('='):  # Ignorer les lignes de séparation
        continue
    tokens = sent.split()
    sentences.append(tokens)

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

Il y a 1228 phrases.
En voici 3 au hasard :
[['brunswick', 'also', 'has', 'interests_5', 'in', 'defense', 'and', 'aerospace', 'products', '.'], ['market', 'analysts', 'said', 'that', 'continued', 'declines', 'and', 'volatility', 'in', 'the', 'stock', 'market', 'contributed', 'generally', 'to', 'enhanced', 'investor', 'interest_1', 'in', 'precious', 'metals', ',', 'but', 'that', 'the', 'british', 'political', 'news', 'was', 'the', 'main', 'factor', 'boosting', 'prices', ':', 'chancellor', 'of', 'the', 'exchequer', 'nigel', 'lawson', 'resigned', '.'], ['but', 'he', 'also', 'noted', 'that', 'if', 'uncertainty', 'continues', 'in', 'the', 'stock', 'market', ',', 'that', 'might', 'create', 'some', 'investor', 'interest_1', 'in', 'precious', 'metals', '.']]


## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [12]:
import random

In [13]:
def wsd_lesk(senses, sentence, window_size=5):
    interest_indices = [i for i, w in enumerate(sentence) if 'interest' in w.lower()]
    
    if not interest_indices:
        raise ValueError("La phrase ne contient pas d'occurrence de 'interest' annoté.")
    
    idx = interest_indices[0]
    
    # Extraire le contexte autour de 'interest' dans la fenêtre définie
    start = max(0, idx - window_size)
    end = min(len(sentence), idx + window_size + 1)
    context_words = set(w.lower().strip('.,;:"\'()[]') for w in sentence[start:end] if 'interest' not in w.lower())
    
    # Calculer le score de chevauchement pour chaque sens
    scores = []
    for sense_idx, sense_words in enumerate(senses, start=1):
        overlap = len(context_words.intersection(sense_words))
        scores.append((sense_idx, overlap))
    
    max_score = max(scores, key=lambda x: x[1])[1]
    
    # Extraire tous les sens avec le score maximum (possibilité d'égalité)
    best_senses = [sense for sense, score in scores if score == max_score]
    
    chosen_sense = random.choice(best_senses)
    
    return chosen_sense

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [12]:
def evaluate_wsd(fct_name, senses, sentences):
    correct = 0
    total = len(sentences)
    
    for sentence in sentences:
        interest_word = next((w for w in sentence if 'interest_' in w.lower() or 'interests_' in w.lower()), None)
        if interest_word is None:
            continue
        
        true_sense = int(interest_word.split('_')[-1].split('/')[0])  # en cas de POS tags
        
        pred_sense = fct_name(senses, sentence)
        
        if pred_sense == true_sense:
            correct += 1
    
    score = (correct / total) * 100 if total > 0 else 0
    return score

**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [15]:
best_score = 0
best_window = 0
best_senses = None

for window in range(1, 11):
    def lesk_with_window(senses, sentence):
        return wsd_lesk(senses, sentence, window_size=window)
    
    score1 = evaluate_wsd(lesk_with_window, senses1, sentences)
    score2 = evaluate_wsd(lesk_with_window, senses2, sentences)
    
    print(f"Window={window}: senses1 score={score1:.2f}%, senses2 score={score2:.2f}%")
    
    if score1 > best_score:
        best_score = score1
        best_window = window
        best_senses = "senses1"
    if score2 > best_score:
        best_score = score2
        best_window = window
        best_senses = "senses2"

print(f"\nMeilleur score : {best_score:.2f}% avec fenêtre {best_window} et {best_senses}")

Window=1: senses1 score=4.48%, senses2 score=21.91%
Window=2: senses1 score=5.21%, senses2 score=23.05%
Window=3: senses1 score=6.35%, senses2 score=23.78%
Window=4: senses1 score=5.78%, senses2 score=23.86%
Window=5: senses1 score=5.86%, senses2 score=24.43%
Window=6: senses1 score=6.03%, senses2 score=23.53%
Window=7: senses1 score=6.35%, senses2 score=24.10%
Window=8: senses1 score=5.70%, senses2 score=24.84%
Window=9: senses1 score=7.00%, senses2 score=22.96%
Window=10: senses1 score=5.86%, senses2 score=23.05%

Meilleur score : 24.84% avec fenêtre 8 et senses2


## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.

In [8]:
# Model is then located at home directory, adjust as needed.
import os
file_path = os.path.join(os.path.expanduser("~"), "gensim-data", "word2vec-google-news-300", "word2vec-google-news-300.gz")

In [9]:
# Takes ~1min to load the model
import gensim
from gensim.models import KeyedVectors
wv_model = gensim.models.KeyedVectors.load_word2vec_format(file_path, binary=True)  # C bin format

In [10]:
def wsd_word2vec(senses, sentence, window_size=5):
    interest_indices = [i for i, w in enumerate(sentence) if 'interest' in w.lower()]

    if not interest_indices:
        raise ValueError("La phrase ne contient pas d'occurrence de 'interest' annoté.")

    idx = interest_indices[0]

    # Extraire le contexte autour de 'interest' dans la fenêtre définie
    start = max(0, idx - window_size)
    end = min(len(sentence), idx + window_size + 1)
    context_words = [w.lower().strip('.,;:"\'()[]') for w in sentence[start:end] if 'interest' not in w.lower()]

    context_words = [w for w in context_words if w in wv_model.key_to_index]

    if not context_words:
        return random.randint(1, len(senses))

    # Calculer le score de chevauchement pour chaque sens
    scores = []
    for sense_idx, sense_words in enumerate(senses, start=1):
        valid_sense_words = [w for w in sense_words if w in wv_model.key_to_index]

        if not valid_sense_words:
            scores.append((sense_idx, 0))
            continue

        try:
            similarity = wv_model.n_similarity(context_words, valid_sense_words)
            scores.append((sense_idx, similarity))
        except (KeyError, ZeroDivisionError):
            scores.append((sense_idx, 0))

    max_score = max(scores, key=lambda x: x[1])[1]

    # Extraire tous les sens avec le score maximum (possibilité d'égalité)
    best_senses = [sense for sense, score in scores if score == max_score]

    chosen_sense = random.choice(best_senses)

    return chosen_sense

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [19]:
best_score = 0
best_window = 0
best_senses = None

for window in range(1, 11):
    def word2vec_with_window(senses, sentence):
        return wsd_word2vec(senses, sentence, window_size=window)

    score1 = evaluate_wsd(word2vec_with_window, senses1, sentences)
    score2 = evaluate_wsd(word2vec_with_window, senses2, sentences)

    print(f"Window={window}: senses1 score={score1:.2f}%, senses2 score={score2:.2f}%")

    if score1 > best_score:
        best_score = score1
        best_window = window
        best_senses = "senses1"
    if score2 > best_score:
        best_score = score2
        best_window = window
        best_senses = "senses2"

print(f"\nMeilleur score : {best_score:.2f}% avec fenêtre {best_window} et {best_senses}")

Window=1: senses1 score=1.63%, senses2 score=53.34%
Window=2: senses1 score=2.36%, senses2 score=55.62%
Window=3: senses1 score=1.55%, senses2 score=57.41%
Window=4: senses1 score=1.55%, senses2 score=57.82%
Window=5: senses1 score=1.55%, senses2 score=58.47%
Window=6: senses1 score=1.47%, senses2 score=59.45%
Window=7: senses1 score=0.90%, senses2 score=57.82%
Window=8: senses1 score=1.47%, senses2 score=57.57%
Window=9: senses1 score=0.98%, senses2 score=56.68%
Window=10: senses1 score=1.14%, senses2 score=55.62%

Meilleur score : 59.45% avec fenêtre 6 et senses2


> Le score est significativement meilleur, doublé, passant de 24.84% avec Lesk à 59.45% avec word2vec


## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [37]:
def extract_features(sentence, target_idx, window_size=2):
    features = {}

    target_word = sentence[target_idx].lower()
    # strips suffixes denoting sense
    if '_' in target_word:
        base_word = target_word.split('_')[0]
    else:
        base_word = target_word
    features['word0'] = base_word

    # window iteration
    for i in range(1, window_size + 1):
        # words before
        pos = target_idx - i
        if pos >= 0:
            features[f'word-{i}'] = sentence[pos].lower()
        else:
            features[f'word-{i}'] = 'NONE'

        # words after
        pos = target_idx + i
        if pos < len(sentence):
            features[f'word+{i}'] = sentence[pos].lower()
        else:
            features[f'word+{i}'] = 'NONE'

    return features


def items_features_per_window(window_size):
    items_features = []
    for sentence in sentences:
        # find occurrences of interest/interests with annotation
        for i, word in enumerate(sentence):
            if 'interest_' in word.lower() or 'interests_' in word.lower():
                # Extract the sense and feature
                sense = int(word.split('_')[-1])
                features = extract_features(sentence, i, window_size)
                items_features.append([features, sense])
    return items_features

In [38]:
window_size = 2
items_with_features = items_features_per_window(window_size)

print(len(items_with_features))
print(items_with_features[151:154])

1228
[[{'word0': 'interests', 'word-1': 'has', 'word+1': 'in', 'word-2': 'also', 'word+2': 'defense'}, 5], [{'word0': 'interest', 'word-1': 'investor', 'word+1': 'in', 'word-2': 'enhanced', 'word+2': 'precious'}, 1], [{'word0': 'interest', 'word-1': 'investor', 'word+1': 'in', 'word-2': 'some', 'word+2': 'precious'}, 1]]


**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [21]:
from random import shuffle

In [39]:
def split_train_test(split_point=0.8):
    train = []
    test  = []
    # Group items by sense
    sense_groups = {}
    for item in items_with_features:
        sense = item[1]
        if sense not in sense_groups:
            sense_groups[sense] = []
        sense_groups[sense].append(item)

    for sense, items in sense_groups.items():
        # Shuffle items by sense
        items_copy = items.copy()
        shuffle(items_copy)

        # Split into train and test
        split_idx = int(len(items_copy) * split_point)
        train.extend(items_copy[:split_idx])
        test.extend(items_copy[split_idx:])

    # Additional shuffle post split
    shuffle(train)
    shuffle(test)
    return train, test

In [40]:
iwf_train, iwf_test = split_train_test()

print(len(iwf_train), ' ', len(iwf_test))
print(iwf_test[:2], iwf_test[-2:])

980   248
[[{'word0': 'interest', 'word-1': 'NONE', 'word+1': 'rates', 'word-2': 'NONE', 'word+2': 'can'}, 6], [{'word0': 'interest', 'word-1': 'and', 'word+1': '.', 'word-2': 'principal', 'word+2': 'NONE'}, 6]] [[{'word0': 'interest', 'word-1': 'u.s.', 'word+1': 'rates', 'word-2': 'that', 'word+2': 'are'}, 6], [{'word0': 'interest', 'word-1': 'no', 'word+1': ';', 'word-2': 'has', 'word+2': 'again'}, 5]]


**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [47]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy

def test_and_train(train, test):
    train_set = [(item[0], item[1]) for item in train]
    test_set = [(item[0], item[1]) for item in test]

    classifier = NaiveBayesClassifier.train(train_set)
    return classifier, accuracy(classifier, test_set)

In [59]:
best_acc = 0
best_window = 0
best_classifier = None
best_iwf_test = None

for window in range(1, 11):
    items_with_features = items_features_per_window(window)
    iwf_train, iwf_test = split_train_test()
    classifier, acc = test_and_train(iwf_train, iwf_test)
    print(f"Accuracy for window {window:2d} : {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_window = window
        best_classifier = classifier
        best_iwf_test = iwf_test

print(f"\nBest window size: {best_window} with accuracy: {best_acc:.4f}")

Accuracy for window  1 : 0.8710
Accuracy for window  2 : 0.8548
Accuracy for window  3 : 0.8508
Accuracy for window  4 : 0.8468
Accuracy for window  5 : 0.8105
Accuracy for window  6 : 0.8548
Accuracy for window  7 : 0.8105
Accuracy for window  8 : 0.8185
Accuracy for window  9 : 0.7903
Accuracy for window 10 : 0.7782

Best window size: 1 with accuracy: 0.8710


> Le meilleur score varie pour une fenêtre de 1 ou de 2.
> Il est néanmoins bien plus élevé que les deux précédentes méthodes en atteignant 85%-89% tandis que word2vec n'atteignait que 59.45% avec une fenêtre de 6.

**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [63]:
best_classifier.show_most_informative_features(20)

Most Informative Features
                  word+1 = 'in'                1 : 6      =     46.5 : 1.0
                  word-1 = 'other'             3 : 6      =     20.3 : 1.0
                  word+1 = 'of'                4 : 6      =     19.9 : 1.0
                  word-1 = 'of'                4 : 5      =     11.2 : 1.0
                  word-1 = 'and'               6 : 5      =     10.2 : 1.0
                  word+1 = '.'                 3 : 6      =     10.1 : 1.0
                  word-1 = 'with'              5 : 6      =      9.5 : 1.0
                  word-1 = 'own'               4 : 6      =      9.1 : 1.0
                  word-1 = 'in'                6 : 5      =      8.7 : 1.0
                  word+1 = 'to'                4 : 6      =      8.0 : 1.0
                  word-1 = 'any'               1 : 6      =      7.6 : 1.0
                  word-1 = 'our'               4 : 6      =      7.1 : 1.0
                  word-1 = 'public'            4 : 1      =      6.0 : 1.0

> Nous observons que le mot suivant 'interest/s' qui permet de déterminer son sens de manière la plus tranchée est 'in'.
> Celui-ci permet de déterminer qu'il a 46.5 fois plus de chances d'être utilisé dans le sens 1 que dans le sens 6
>
> À noter que 'interest' se trouve également dans cette liste, en posittion 0, ce qui est attendu.
> Ce qui l'est moins est que celui-ci est 4.8 fois plus souvent associé au sens 6 que 3.

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [62]:
def evaluate_wsd_supervised(classifier, items_with_features):
    total_by_sense = {}
    correct_by_sense = {}

    # Process each test item
    for item in items_with_features:
        features = item[0]
        true_sense = item[1]
        predicted_sense = classifier.classify(features)

        # Update counters
        if true_sense not in total_by_sense:
            total_by_sense[true_sense] = 0
            correct_by_sense[true_sense] = 0

        total_by_sense[true_sense] += 1
        if predicted_sense == true_sense:
            correct_by_sense[true_sense] += 1

    # Calculate accuracy for each sense
    print("Accuracy by sense:")
    overall_correct = 0
    overall_total = 0

    for sense in sorted(total_by_sense.keys()):
        accuracy = (correct_by_sense[sense] / total_by_sense[sense]) * 100
        print(f"Sense {sense}: {accuracy:.2f}% ({correct_by_sense[sense]}/{total_by_sense[sense]})")
        overall_correct += correct_by_sense[sense]
        overall_total += total_by_sense[sense]

    # Overall accuracy
    overall_accuracy = (overall_correct / overall_total) * 100
    print(f"\nOverall accuracy: {overall_accuracy:.2f}% ({overall_correct}/{overall_total})")
    return overall_accuracy


evaluate_wsd_supervised(best_classifier, iwf_test)

Accuracy by sense:
Sense 1: 94.12% (32/34)
Sense 2: 0.00% (0/1)
Sense 3: 37.50% (3/8)
Sense 4: 86.96% (20/23)
Sense 5: 83.33% (35/42)
Sense 6: 97.86% (137/140)

Overall accuracy: 91.53% (227/248)


91.53225806451613

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

> | Méthode   | Score  | Fenêtre |
> |-----------|--------|---------|
> | Lesk      | 24.84% | 8       |
> | Word2vec  | 59.45% | 6       |
> | Supervisé | 91.53% | 1-2     |

> Le meilleur score obtenu est sans surprise à l'aide de la méthode supervisée.
> En effet les 2 autres sont des méthodes automatisées moins élaborées sans le fine tuning que la supervision nous offre.